In [6]:
# First Cell: Install required libraries (if not already installed)
!pip install mysql-connector-python openai langchain


In [7]:
# Second Cell: Import required libraries
import mysql.connector
from openai import OpenAI
import openai
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [8]:
# Third Cell: Connect to MySQL Database
def get_data_from_mysql(query):
    """
    Connects to MySQL database and executes a query.
    Returns the result of the query.
    """
    conn = mysql.connector.connect(
        host="localhost",  # Replace with your MySQL host (localhost is default)
        user="your_user",  # Replace with your MySQL username
        password="your_password",  # Replace with your MySQL password
        database="sample_db"  # Replace with your database name
    )
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    return result


In [9]:
# Fourth Cell: Set up OpenAI API Key (or any other LLM model like Ollama)
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your OpenAI API Key


In [10]:
# Fifth Cell: Define Prompt Template for LangChain
prompt_template = """
You are a helpful assistant. Based on the following query, first generate an appropriate SQL query to extract relevant data from the database, then return the answer in natural language as if you're speaking to a person.
Query: {query}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["query"])


In [11]:
# Sixth Cell: Initialize the LLM (Large Language Model) for Query Processing
llm = OpenAI(model="text-davinci-003")  # Use "gpt-3.5-turbo" or another model as per your needs


TypeError: OpenAI.__init__() got an unexpected keyword argument 'model'

In [ ]:
# Seventh Cell: Create LangChain to Process Queries
chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
# Eighth Cell: Function to Process Natural Language Queries
def process_query(natural_query):
    """
    Takes a natural language query, generates the corresponding SQL query,
    fetches the data from the MySQL database, and then returns the answer in natural language.
    """
    # Generate the SQL query from the natural query using LangChain
    sql_query = chain.run(natural_query)
    print(f"Generated SQL Query: {sql_query}")
    
    # Now fetch the data from MySQL using the generated SQL
    data = get_data_from_mysql(sql_query)
    
    # Translate the result back into natural language for the user
    if not data:
        return "I couldn't find any information related to your query."
    
    # Formatting data as a natural language response
    if len(data) == 1:
        # If only one result is returned
        return f"The answer to your query is: {data[0]}"
    else:
        # If multiple rows are returned, summarize the information
        response = "Here is the information I found:\n"
        for row in data:
            response += str(row) + "\n"
        return response


In [ ]:
# Ninth Cell: Example MySQL Data (Run this once if you need sample data in MySQL)
def setup_mysql_data():
    """
    This function sets up sample data in MySQL if needed.
    Run this once to populate the database.
    """
    conn = mysql.connector.connect(
        host="localhost",
        user="your_user",
        password="your_password",
        database="sample_db"
    )
    cursor = conn.cursor()
    
    # Create a sample table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS customers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        age INT,
        city VARCHAR(255)
    );
    """)
    
    # Insert sample data into the table
    cursor.execute("INSERT IGNORE INTO customers (name, age, city) VALUES ('Alice', 30, 'New York');")
    cursor.execute("INSERT IGNORE INTO customers (name, age, city) VALUES ('Bob', 25, 'San Francisco');")
    
    conn.commit()
    conn.close()

# Uncomment to set up data (Run only once)
# setup_mysql_data()
# Ninth Cell: Example MySQL Data (Run this once if you need sample data in MySQL)
def setup_mysql_data():
    """
    This function sets up sample data in MySQL if needed.
    Run this once to populate the database.
    """
    conn = mysql.connector.connect(
        host="localhost",
        user="your_user",
        password="your_password",
        database="sample_db"
    )
    cursor = conn.cursor()
    
    # Create a sample table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS customers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        age INT,
        city VARCHAR(255)
    );
    """)
    
    # Insert sample data into the table
    cursor.execute("INSERT IGNORE INTO customers (name, age, city) VALUES ('Alice', 30, 'New York');")
    cursor.execute("INSERT IGNORE INTO customers (name, age, city) VALUES ('Bob', 25, 'San Francisco');")
    
    conn.commit()
    conn.close()

# Uncomment to set up data (Run only once)
# setup_mysql_data()


In [ ]:
# Tenth Cell: Example Query - Process Natural Language to SQL and Get Answer
query = "What is the age of Alice?"
answer = process_query(query)
print(f"Answer: {answer}")
